# Leaflet web map of Last.fm artists

Convert the geocoded CSV file of artists produced by [musicbrainz_geocoder.ipynb](musicbrainz_geocoder.ipynb) to a GeoJS

In [9]:
import pandas as pd, json
from IPython.display import IFrame

In [2]:
# load the geocoded data set of artists
df = pd.read_csv('data/mb_geocoded.csv', encoding='utf-8')
print '{:,} total rows'.format(len(df))
df = df[pd.notnull(df['place_latlng'])]
print '{:,} rows with lat-long'.format(len(df))
print '{:,} unique lat-longs'.format(len(df['place_latlng'].unique()))

12,507 total rows
5,851 rows with lat-long
1,931 unique lat-longs


In [3]:
# determine how many times each place appears in dataset, and break latlng into discrete lat and long
place_counts = df['place_full'].value_counts()
df['place_count'] = df['place_full'].map(lambda x: place_counts[x])
df['lat'] = df['place_latlng'].map(lambda x: x.split(',')[0])
df['lng'] = df['place_latlng'].map(lambda x: x.split(',')[1])
df = df[['name', 'place_full', 'place_count', 'lat', 'lng']]
df.head()

,name,place_full,place_count,lat,lng
3,The Temptations,"Detroit, Wayne County, Michigan, United States",65,42.3486635,-83.0567374
4,Nick Cave & The Bad Seeds,"Melbourne, Victoria, Australia",41,-37.8142175,144.9631608
6,Pink Floyd,"London, England, United Kingdom",238,51.5073219,-0.1276473
8,Houndstooth,"Portland, Multnomah County, Oregon, United States",41,45.5202471,-122.6741948
9,Built to Spill,"Boise, Ada County, Idaho, United States",4,43.61656,-116.2008349


In [4]:
# create html list of artists from each place
features = []
for place_full in df['place_full'].unique():
    
    # how many artists to show before saying "...and n more"
    num_to_show = 3
    line_break = u'<br />'
    artists = u''
    
    place_count = place_counts[place_full]
    names = df[df['place_full']==place_full]['name']
    
    if place_count <= num_to_show:
        for name in names:
            artists = u'{}{}{}'.format(artists, name, line_break)
            
    else:
        for name in names[0:num_to_show]:
            artists = u'{}{}{}'.format(artists, name, line_break)
        artists = u'{}...and {} more'.format(artists, place_count - num_to_show)
        
    features.append([place_full, artists])

df_leaflet = pd.DataFrame(features, columns=['place_full', 'artists'])

In [5]:
# add lat and long back to the dataframe
place_lat_lng = {}
df_unique = df[['place_full', 'lat', 'lng']].drop_duplicates(subset='place_full')
for label in df_unique.index:
    place_lat_lng[df_unique.loc[label, 'place_full']] = (df_unique.loc[label, 'lat'], df_unique.loc[label, 'lng'])
    
df_leaflet['lat'] = df_leaflet['place_full'].map(lambda x: place_lat_lng[x][0])
df_leaflet['lng'] = df_leaflet['place_full'].map(lambda x: place_lat_lng[x][1])
df_leaflet.head()

,place_full,artists,lat,lng
0,"Detroit, Wayne County, Michigan, United States",The Temptations<br />Danny Brown<br />Gore Gor...,42.3486635,-83.0567374
1,"Melbourne, Victoria, Australia",Nick Cave & The Bad Seeds<br />Eddy Current Su...,-37.8142175,144.9631608
2,"London, England, United Kingdom",Pink Floyd<br />Erasure<br />Roxy Music<br />....,51.5073219,-0.1276473
3,"Portland, Multnomah County, Oregon, United States",Houndstooth<br />Wipers<br />Blitzen Trapper<b...,45.5202471,-122.6741948
4,"Boise, Ada County, Idaho, United States",Built to Spill<br />Eilen Jewell<br />Paul Rev...,43.61656,-116.2008349


## Write to GeoJSON for leaflet mapping

In [6]:
# function to write the dataframe out to geojson
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}

    # loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        # create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}

        # fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat]]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
        
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    
    return geojson

In [7]:
geojson = df_to_geojson(df_leaflet, df_leaflet.columns, lat='lat', lon='lng')

# save the geojson result to a file
output_filename = 'leaflet/lastfm-dataset.js'
with open(output_filename, 'wb') as output_file:
    output_file.write('var dataset = {};'.format(json.dumps(geojson)))
    
# how many features did we save to the geojson file?
print '{:,} geotagged features saved to file'.format(len(geojson['features']))

1,951 geotagged features saved to file


To see this live, visit: http://geoffboeing.com/2016/05/analyzing-lastfm-history/#more-1799

In [10]:
# show the iframe of the leaflet web map here
IFrame('leaflet/lastfm-artists-map.html', width=600, height=400)